## Pip installs

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install pysam

In [ ]:
!pip install HTSeq

## Loading DNA sequence data from BAMs to create dataset for fine tuning DNABERT

### Import stuff

In [2]:
import HTSeq
import pysam
from collections import defaultdict

### Function definitions

In [ ]:
def seq2kmer_label(seq, k):
    """
    Convert original sequence to kmers, also labels data by the position of the mutation in the kmer 
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers


In [29]:
def addBAM(bam_path, vcf_path):
    '''
        Given a path pointing ot a bam file, return a json or hf dataset object with following info:
            1. iterate through the bam file and get sequence info on reads containing mutations appearing in vcf file
            2. for each read want to have the chr #, description, and whther it is in exonic portion of a gene
                -if not exonic, ideally want what type non-coding element the read is a part of
            3. If a mutation, want info for Mutation from Clinvar, Cosmic, NCBI
            4. desc of gene from NCBI 
            
            desc of sam_alignment from htseq:
                >>> aln.iv
                <GenomicInterval object 'IV', [246048,246084), strand '+'>
                >>> aln.iv.chrom
                'IV'
                >>> aln.iv.start
                246048
                >>> aln.iv.end
                246084
                >>> aln.iv.strand
                '+'
    '''
    data_dict = {}
    with HTSeq.BAM_Reader(bam_path) as f:
        for i, sam_alignment in enumerate(f):
            ### did this a alittle backwards, should iterate through gtf file to get gene name and interval, then (hopefully) use that
            ### to index into the bam file to get the sequence rather than iterating through all parts of the bam file
            if sam_alignment.aligned == True:
                ### i'm not sure what's gonna be the most useful thing for llama to map all the info to, starting with a string with
                ### chrom and pos and what file it's from
                chrom_pos_identifier = sam_alignment.iv.chrom + ' START: ' + str(sam_alignment.iv.start) + ' END: ' + str(sam_alignment.iv.end) 
                data_dict[chrom_pos_identifier] = [
                {
                    'read_name':sam_alignment.read.name,
                    'seq':sam_alignment.read
                }
                ]
            print(data_dict)
            ## for testing don't do the whole thing
            if i == 5:
                break

In [ ]:
def prepareBertDataset(bam_file, vcf_file):
    '''
        get mutations from vcf file, then get the read containing that mutation from the bam file, then mask that mutation and format
        for BERT (i.e. token with the mask at the mutation position so it can learn with PTA artifacts look like
        
        dataset should look like this:
        DatasetDict({
                train: Dataset({
                    features: ['squence', 'label'],
                    num_rows: 25000
                })
                test: Dataset({
                    features: ['sequence', 'label'],
                    num_rows: 25000
                })
                unsupervised: Dataset({
                    features: ['sequence', 'label'],
                    num_rows: 50000
                })
            })
        
        for row in sample:
            print(f"\n'>>> Sequence: {row['sequence']}'")
            print(f"'>>> Label: {row['label']}'")

        '>>> Sequence: ACTAGATAGATA'
        '>>> Label: 0'
                            
        '>>> Review: ACATAGATATATA'
                                ^
                            mutated base
        '>>> Label: 1'
        
        ### If we do this the model will only learn whether a mutation is in the sequence, can we label it so the number corresponds to
        ### what base is mutated, i.e. 0 is no mutation?
        
        '>>> Sequence: ACTAGATAGATA'
        '>>> Label: 0'
                            
        '>>> Review: ACATAGATATATA'
                                ^
                            mutated base
        '>>> Label: 12'
        
        ### this also seems like it won't leverage the power of the whole dna sequence
        
    '''
    
    import pysam

    bamfile = pysam.AlignmentFile("your_bam_file.bam", "rb")
    reads = bamfile.fetch("chromosome_name", start_position, end_position
    

In [17]:
## variable definitions
bam_dir='/scratch/users/sschulz/pta_on_normal'
bam_file='CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B8_S51.realigned_deduped_sorted.bam'
bam_path = bam_dir + '/' + bam_file
vcf_name='CARTPt04_Scan2_svc_merged_extract_snp.vcf'
vcf_path = bam_dir + '/' + vcf_name

In [54]:
bamfile = pysam.AlignmentFile(bam_path, "rb")
reads = bamfile.fetch("chr1", 10, 100000)

In [49]:
bamfile

In [65]:
samfile=bamfile
pileup = samfile.pileup('chr1', 1000, 20000)
for x in pileup:
    print(str(x))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



0	18393	15
VH00736:15:AAAVYTKHV:2:2305:35197:10580	99	#0	18296	0	100M	#0	18433	237	TTCTCAATCTTGGCCTGGGCCAAGGAGACCTTCTCTCCAATGGCCTGCACCTGGCTCCGGCTCTGCTCTACCTGCTGGGAGATCCTGCCATGGAGAAGAT	array('B', [34, 34, 26, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 26, 34, 34, 34, 34, 26, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 26, 34, 34, 34, 12, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 26, 34, 34, 34, 34, 34, 34, 34, 34, 12, 34, 26, 26, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 26, 26, 34])	[('NM', 0), ('MD', '100'), ('AS', 100), ('XS', 100), ('RG', 'CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B8_S51'), ('XA', 'chr1,+188819,100M,0;chr9,+18409,100M,0;chrX,-156021665,100M,0;chr2,-113595034,100M,0;chr15,-101972567,100M,0;chr16,+17985,100M,0;chr12,+18401,100M,0;chr12_GL877875v1_alt,+8401,100M,0;'), ('MC', '100M'), ('MQ', 0)]	98	0	-2144205563	0	0	0	0
VH00736:15:AAAVYTKHV:2:1205:48983:48820	99	

In [64]:
pileup

## DNA tokenizer Test


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("AIRI-Institute/gena-lm-bert-base")


In [ ]:
encoded_input = tokenizer("ACGTGGTATGATGATAGATGATGA")


In [ ]:
print(encoded_input)

In [ ]:
tokenizer.decode(encoded_input["input_ids"])

In [ ]:
batch_sequences = [
    "ACGTAGCTGACTGACTTAGTGA",
    "ACTAGCATGCATCGTAGCTAGCTAGACTGA",
    "ATATATATTACACACACGAGACTAGCTT",
]

In [ ]:
encoded_input=tokenizer(batch_sequences, padding=True, truncation=True)

In [ ]:
print(encoded_input)

In [ ]:
for i in encoded_input['input_ids']:
    print(tokenizer.decode(i))

In [ ]:
## p much above but we've padded, truncated (no maximum length provided tho) and returned tensors
encoded_input = tokenizer(batch_sequences, padding=True, truncation=True, return_tensors="pt")

In [ ]:
## so we can tokenize DNA sequences, but how do we 